# Celestial Body Database Project
## FreeCodeCamp Course
### Relational Database
The necessary steps for this project to pass are outlined here: <https://www.freecodecamp.org/learn/relational-database/build-a-celestial-bodies-database-project/build-a-celestial-bodies-database>

I wanted to do some additional things for this project and so I did some searching and webscraping to populate my tables. 

In [1]:
import requests
import pandas as pd
import numpy as np
from io import StringIO

### Galaxies
The process is more or less the same for every table. pandas scrapes the html and saves the tables into an array. I select the right one out of this array and manipulate it to fit the project specs.

In [5]:
url_galaxies = 'https://en.wikipedia.org/wiki/List_of_galaxies'
request_galaxies = requests.get(url_galaxies)
dfs_galaxies = pd.read_html(StringIO(request_galaxies.text))
galaxies = dfs_galaxies[0]
galaxies = galaxies[['Galaxy', 'Constellation', 'Origin of name', 'Notes']]
galaxies.columns = ['name', 'constellations', 'description', 'notes']
galaxies.loc[len(galaxies)] = ['Unknown', '', '', '']
galaxies.insert(0, "galaxy_id", range(len(galaxies)))

/tmp/ipykernel_3554/1660544688.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  galaxies.loc[len(galaxies)] = ['Unknown', '', '', '']


In [7]:
galaxies.tail()

,galaxy_id,name,constellations,description,notes
39,39,Sunflower Galaxy,Canes Venatici,Similar in appearance to a sunflower.,NaN
40,40,Tadpole Galaxy,Draco,The name comes from the resemblance of the gal...,This shape resulted from tidal interaction tha...
41,41,Triangulum Galaxy,Triangulum,Named after its location within the Triangulum...,NaN
42,42,Whirlpool Galaxy,Canes Venatici,From the whirlpool appearance this gravitation...,NaN
43,43,Unknown,,,


### Constellations

In [8]:
url_constellations = 'https://starchild.gsfc.nasa.gov/docs/StarChild/questions/88constellations.html'
request_constellations = requests.get(url_constellations)
dfs_constellations = pd.read_html(StringIO(request_constellations.text))
constellations = dfs_constellations[0]
constellations.columns = ['name', 'description']
constellations.insert(0, 'constellation_id', range(len(constellations)))
constellations.head()

,constellation_id,name,description
0,0,Andromeda,Princess of Ethiopia
1,1,Antlia,Air pump
2,2,Apus,Bird of Paradise
3,3,Aquarius,Water bearer
4,4,Aquila,Eagle


In [9]:
gal_dict = {row['constellations']: row['galaxy_id'] for i, row in galaxies.iterrows()}
gal_dict

{'Andromeda': 0,
 'Corvus': 1,
 'Centaurus': 2,
 'Lynx': 17,
 'Coma Berenices': 30,
 'Ursa Major': 34,
 'Virgo': 36,
 'Sculptor': 35,
 'Circinus': 9,
 'Pavo': 12,
 'Sextans': 13,
 'Eridanus': 14,
 'Canes Venatici': 42,
 'Cygnus and Cepheus': 16,
 'Serpens Caput': 19,
 'Dorado/Mensa': 20,
 'Volans': 21,
 'Pegasus': 32,
 'Tucana': 27,
 'Sagittarius (centre)': 29,
 'Cetus': 31,
 'Hydra': 37,
 'Boötes': 38,
 'Draco': 40,
 'Triangulum': 41,
 '': 43}

In [10]:
gal_dict = {row['constellations']: row['galaxy_id'] for i, row in galaxies.iterrows()}
gal_dict.update({ 'Cygnus': 16, 'Cepheus': 16, 'Dorado': 20,'Mensa': 20, 'Sagittarius': 29,'Serpens': 19})
constellations["galaxy_id"] = constellations.apply(lambda x: gal_dict[x['name']] if x['name'] in gal_dict else 43, axis=1)
constellations.head()

,constellation_id,name,description,galaxy_id
0,0,Andromeda,Princess of Ethiopia,0
1,1,Antlia,Air pump,43
2,2,Apus,Bird of Paradise,43
3,3,Aquarius,Water bearer,43
4,4,Aquila,Eagle,43


### Stars
This is a test for one star. In the next step I loop through all the linked wikipedia articles. Vulpecula is a special case, where I have to grab a different table. All the tables have a legend at the end, that I have to scrap.

In [11]:
url_stars = 'https://en.wikipedia.org/wiki/List_of_stars_in_Circinus'
request_stars = requests.get(url_stars)
dfs_stars = pd.read_html(StringIO(request_stars.text))
dfs_stars[0].head()

,Name,B,Var,HD,HIP,RA,Dec,vis. mag.,abs. mag.,Dist. (ly),Sp. class,Notes,Unnamed: 12,Unnamed: 13
0,α Cir,α,NaN,128898,71908,14h 42m 30.69s,−64° 58′ 28.5″,3.18,2.11,53,F1Vp,"rapidly oscillating Ap star, Vmax = 3.17m, Vmi...",NaN,NaN
1,β Cir,β,NaN,135379,74824,15h 17m 30.96s,−58° 48′ 03.2″,4.07,1.71,97,A3Va,has a planet (b),NaN,NaN
2,γ Cir,γ,NaN,136415,75323,15h 23m 22.66s,−59° 19′ 14.5″,4.48,−1.49,509,B5III + F8,"γ Cas variable, Vmax = 4.43m, Vmin = 4.52m; do...",NaN,NaN
3,ε Cir,ε,NaN,135291,74837,15h 17m 38.89s,−63° 36′ 37.8″,4.85,−0.44,373,K2.5III,NaN,NaN,NaN
4,δ Cir,δ,NaN,135240,74778,15h 16m 56.90s,−60° 57′ 26.1″,5.04,−6.42,6392,O8.5V,"Algol variable, Vmax = 5.04m, Vmin = 5.20m, P ...",NaN,NaN


In [12]:
dfs_stars = []
for constellation in constellations['name']:
    request = requests.get('https://en.wikipedia.org/wiki/List_of_stars_in_' + constellation)
    dfs = pd.read_html(StringIO(request.text))
    df = dfs[1 if constellation == "Vulpecula" else 0][:-2]
    df['constellation'] = constellation
    dfs_stars.append(df)

/tmp/ipykernel_3554/4167174005.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['constellation'] = constellation
/tmp/ipykernel_3554/4167174005.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['constellation'] = constellation
/tmp/ipykernel_3554/4167174005.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [15]:
stars = pd.concat(dfs_stars, axis=0, ignore_index=True)
stars.columns

Index(['Name', 'B', 'F', 'Var', 'HD', 'HIP', 'RA', 'Dec', 'vis. mag.',
       'abs. mag.', 'Dist. (ly)', 'Sp. class', 'Notes', 'Unnamed: 13',
       'constellation', 'Unnamed: 12', 'G.', 'Dist.(ly)', 'G.[1]'],
      dtype='object')

In [16]:
cols = ["Name", "vis. mag.", "Sp. class", "constellation"]
stars = stars[cols]
stars.columns = ["name", "brightness_apparent", "stellar_classification", "constellation"]
stars.insert(0, "star_id", range(len(stars)))
stars["galaxy_id"] = stars.apply(lambda x: gal_dict[x['constellation']] if x['constellation'] in gal_dict else 43, axis=1)

In [17]:
name_new = stars.name.apply(lambda x: "unnamed" if x == "NaN" else x)

In [18]:
stars["name"] = name_new

In [19]:
stars.loc[len(stars)] = [len(stars), "Sun", "NaN", "NaN", "None", 43]

stars['brightness_apparent'] = pd.to_numeric(pd.Series(stars.brightness_apparent), errors='coerce')
stars.tail()

,star_id,name,brightness_apparent,stellar_classification,constellation,galaxy_id
11663,11663,PSR B1953+29,NaN,NaN,Vulpecula,43
11664,11664,PSR B2020+28,NaN,NaN,Vulpecula,43
11665,11665,PSR J2007+2722,NaN,NaN,Vulpecula,43
11666,11666,V407 Vul,NaN,NaN,Vulpecula,43
11667,11667,Sun,NaN,NaN,None,43


In [20]:
temp = stars.apply(lambda x: "unnamed" + str(x["star_id"]) if x["name"] == "unnamed" else x["name"], axis=1)
stars["name"] = temp
stars[45:55]

,star_id,name,brightness_apparent,stellar_classification,constellation,galaxy_id
45,45,6 Per,5.31,G8III:var,Andromeda,0
46,46,62 And,5.31,A1V,Andromeda,0
47,47,18 And,5.35,B9V,Andromeda,0
48,48,55 And,5.42,K1III,Andromeda,0
49,49,11 And,5.44,K0III,Andromeda,0
50,50,NaN,5.45,G5III,Andromeda,0
51,51,36 And,5.46,K1IV,Andromeda,0
52,52,V340 And,5.55,A1III,Andromeda,0
53,53,PZ And,5.57,B9p Si,Andromeda,0
54,54,47 And,5.60,A1m,Andromeda,0


In [21]:
stars = stars.drop_duplicates(subset=["name"])
stars.name.value_counts()

name
Alpheratz     1
115 G. Oph    1
185 G. Oph    1
77 G. Oph     1
28 Sco        1
             ..
HD 175395     1
HD 175529     1
HD 167756     1
κ1 CrA        1
Sun           1
Name: count, Length: 10624, dtype: int64

### Planets

In [22]:
url_planets = 'https://nssdc.gsfc.nasa.gov/planetary/factsheet/'
request_planets = requests.get(url_planets)
dfs_planets = pd.read_html(StringIO(request_planets.text))
planets = dfs_planets[0].transpose()
planets.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,NaN,Mass (1024kg),Diameter (km),Density (kg/m3),Gravity (m/s2),Escape Velocity (km/s),Rotation Period (hours),Length of Day (hours),Distance from Sun (106 km),Perihelion (106 km),...,Orbital Velocity (km/s),Orbital Inclination (degrees),Orbital Eccentricity,Obliquity to Orbit (degrees),Mean Temperature (C),Surface Pressure (bars),Number of Moons,Ring System?,Global Magnetic Field?,NaN
1,MERCURY,0.330,4879,5429,3.7,4.3,1407.6,4222.6,57.9,46.0,...,47.4,7.0,0.206,0.034,167,0,0,No,Yes,MERCURY
2,VENUS,4.87,12104,5243,8.9,10.4,-5832.5,2802.0,108.2,107.5,...,35.0,3.4,0.007,177.4,464,92,0,No,No,VENUS
3,EARTH,5.97,12756,5514,9.8,11.2,23.9,24.0,149.6,147.1,...,29.8,0.0,0.017,23.4,15,1,1,No,Yes,EARTH
4,MOON,0.073,3475,3340,1.6,2.4,655.7,708.7,0.384*,0.363*,...,1.0*,5.1,0.055,6.7,-20,0,0,No,No,MOON


In [23]:
planets.columns= [               'name' ,                 'Mass (1024kg)',
                       'diameter_in_km',               'Density (kg/m3)',
                      'gravity_in_m/s^2',        'Escape Velocity (km/s)',
             'Rotation Period (hours)',         'length_of_day_in_hours',
          'Distance from Sun (106 km)',           'Perihelion (106 km)',
                   'Aphelion (106 km)',         'Orbital Period (days)',
             'Orbital Velocity (km/s)', 'Orbital Inclination (degrees)',
                'Orbital Eccentricity',  'Obliquity to Orbit (degrees)',
                'mean_temperature_in_celsius',       'Surface Pressure (bars)',
                     'Number of Moons',                  'has_ring_system',
              'Global Magnetic Field?',                             '']

In [24]:
planets = planets[['name', 'diameter_in_km', 'gravity_in_m/s^2', 'length_of_day_in_hours', 'mean_temperature_in_celsius', 'has_ring_system']]

In [25]:
planets = planets.drop(0)
planets = planets.drop(4)

In [26]:
planets['name'] = planets['name'].apply(str.capitalize)

In [27]:
planets.insert(0, "planet_id", range(len(planets)))
planets["visited_by_men"] = False

In [28]:
planets['star_id'] = 11667

In [29]:
planets.loc[3, 'visited_by_men'] = True

In [30]:
planets

,planet_id,name,diameter_in_km,gravity_in_m/s^2,length_of_day_in_hours,mean_temperature_in_celsius,has_ring_system,visited_by_men,star_id
1,0,Mercury,4879,3.7,4222.6,167,No,False,11667
2,1,Venus,12104,8.9,2802.0,464,No,False,11667
3,2,Earth,12756,9.8,24.0,15,No,True,11667
5,3,Mars,6792,3.7,24.7,-65,No,False,11667
6,4,Jupiter,142984,23.1,9.9,-110,Yes,False,11667
7,5,Saturn,120536,9.0,10.7,-140,Yes,False,11667
8,6,Uranus,51118,8.7,17.2,-195,Yes,False,11667
9,7,Neptune,49528,11.0,16.1,-200,Yes,False,11667
10,8,Pluto,2376,0.7,153.3,-225,No,False,11667


### Moons

In [31]:
url_moons = 'https://littleastronomy.com/moon-names/'
request_moons = requests.get(url_moons)
dfs_moons = pd.read_html(StringIO(request_moons.text))

In [32]:
moons = dfs_moons[0]

In [33]:
moons

,Name,Parent planet,Name meaning
0,Moon,Earth,“month”
1,Phobos,Mars,Greek god of fear and panic
2,Deimos,Mars,Greek god of dread and terror
3,Io,Jupiter,"Ancestor to Perseus, Cadmus, Heracles, and Minos"
4,Europa,Jupiter,Mother of king Minos of Crete
...,...,...,...
212,Psamathe,Neptune,One of the Nereids in Greek mythology
213,Sao,Neptune,One of the Nereids. Associated with “safety” a...
214,Laomedeia,Neptune,One of the Nereids in Greek mythology
215,Neso,Neptune,One of the Nereids in Greek mythology


In [34]:
moons.columns = ["name", "planet", "description"]

In [35]:
moons['visited_by_men'] = False
moons.loc[0, 'visited_by_men'] = True

In [36]:
moons.head()

,name,planet,description,visited_by_men
0,Moon,Earth,“month”,True
1,Phobos,Mars,Greek god of fear and panic,False
2,Deimos,Mars,Greek god of dread and terror,False
3,Io,Jupiter,"Ancestor to Perseus, Cadmus, Heracles, and Minos",False
4,Europa,Jupiter,Mother of king Minos of Crete,False


In [37]:
dict_moons = {row['name']: row['planet_id'] for i, row in planets.iterrows()}
dict_moons

{'Mercury': 0,
 'Venus': 1,
 'Earth': 2,
 'Mars': 3,
 'Jupiter': 4,
 'Saturn': 5,
 'Uranus': 6,
 'Neptune': 7,
 'Pluto': 8}

In [38]:
moons["planet_id"] = moons.apply(lambda x: dict_moons[x['planet']], axis=1)

In [39]:
moons

,name,planet,description,visited_by_men,planet_id
0,Moon,Earth,“month”,True,2
1,Phobos,Mars,Greek god of fear and panic,False,3
2,Deimos,Mars,Greek god of dread and terror,False,3
3,Io,Jupiter,"Ancestor to Perseus, Cadmus, Heracles, and Minos",False,4
4,Europa,Jupiter,Mother of king Minos of Crete,False,4
...,...,...,...,...,...
212,Psamathe,Neptune,One of the Nereids in Greek mythology,False,7
213,Sao,Neptune,One of the Nereids. Associated with “safety” a...,False,7
214,Laomedeia,Neptune,One of the Nereids in Greek mythology,False,7
215,Neso,Neptune,One of the Nereids in Greek mythology,False,7


In [40]:
moons.insert(0, 'moon_id', range(len(moons)))

In [41]:
constellations.to_csv('constellation.csv', index=False)
galaxies.to_csv('galaxy.csv', index=False)
stars.to_csv('star.csv', index=False)
planets.to_csv('planet.csv', index=False)
moons.to_csv('moon.csv', index=False)